In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# CDF Response Time Analysis Example 2
# Demonstrates class switching in a closed network with 3 classes

model = Network('model')

# Create closed queueing network with Delay and Queue nodes
node = [None] * 2  # using 0-based indexing
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue2', SchedStrategy.PS)

# Three closed classes - only Class1 has initial population
jobclass = [None] * 3  # using 0-based indexing
jobclass[0] = ClosedClass(model, 'Class1', 1, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 0, node[0], 0)
jobclass[2] = ClosedClass(model, 'Class3', 0, node[0], 0)

# Class1 doesn't complete (stays in the system for class switching)
jobclass[0].completes = False

# Service processes at Delay node
node[0].set_service(jobclass[0], Exp(1/1))  # rate = 1
node[0].set_service(jobclass[1], Exp(1/1))  # rate = 1
node[0].set_service(jobclass[2], Exp(1/1))  # rate = 1

# Service processes at Queue node
node[1].set_service(jobclass[0], Exp(1/1))      # rate = 1
node[1].set_service(jobclass[1], Erlang(1/2, 2)) # Erlang with rate=1/2, order=2
node[1].set_service(jobclass[2], Exp(1/0.01))   # rate = 100

# Complex routing matrix with class switching
P = model.init_routing_matrix()

# Class 1 to Class 1: Delay->Queue2
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)

# Class 1 to Class 2: Queue2->Delay (class switch)
P.set(jobclass[0], jobclass[1], node[1], node[0], 1.0)

# Class 2 to Class 1: Delay->Queue2 (class switch back)
P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)

# Class 2 to Class 2: Delay->Queue2
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)

# Class 3 circulates within itself
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)
P.set(jobclass[2], jobclass[2], node[1], node[0], 1.0)

model.link(P)

In [3]:
# NOTE: The JAR test shows that JMT getCdfRespT is not implemented for multi-class models
# So we only implement the Fluid solver scenario that works

# Solve with Fluid solver - aligned with JAR test scenario
options = SolverFluid.defaultOptions()
options.method = 'statedep'  # Aligned with JAR test: method="statedep"
options.iter_max = 100  # Aligned with JAR test: iter_max=100

solver = SolverFluid(model, options)
AvgRespT = solver.get_avg_resp_t()
print('AvgRespT =')
print(AvgRespT)

# Get CDF of response times
FC = solver.get_cdf_resp_t()

# Calculate statistics from CDF
AvgRespTfromCDF = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
PowerMoment2_R = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
Variance_R = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))
SqCoeffOfVariationRespTfromCDF = np.zeros((model.get_number_of_stations(), model.get_number_of_classes()))

for i in range(model.get_number_of_stations()):
    for c in range(model.get_number_of_classes()):
        if FC[i][c] is not None and len(FC[i][c]) > 1:
            # Calculate mean from CDF
            diffs = np.diff(FC[i][c][:, 0])
            values = FC[i][c][1:, 1]
            AvgRespTfromCDF[i, c] = np.sum(diffs * values)
            
            # Calculate second moment and variance
            PowerMoment2_R[i, c] = np.sum(diffs * (values ** 2))
            Variance_R[i, c] = PowerMoment2_R[i, c] - AvgRespTfromCDF[i, c] ** 2
            SqCoeffOfVariationRespTfromCDF[i, c] = Variance_R[i, c] / (AvgRespTfromCDF[i, c] ** 2)

print('AvgRespTfromCDF =')
print(AvgRespTfromCDF)
print('SqCoeffOfVariationRespTfromCDF =')
print(SqCoeffOfVariationRespTfromCDF)

# Commented out JMT solver scenario - JAR test shows it's disabled for multi-class CDF models
# # Solve with JMT solver - aligned with JAR test scenario (disabled in JAR)
# # solver = SolverJMT(model, seed=23000, samples=10000)
# # FC_jmt = solver.get_cdf_resp_t()
# # print("JMT solver CDF analysis would be here, but it's not implemented for multi-class models")

[[1. 1. 0.]
 [1. 4. 0.]]
AvgRespT =
[[1. 1. 0.]
 [1. 4. 0.]]
INFO: Added 20 refined points between t=0.000000 and t=0.002001
INFO: Added 20 refined points between t=0.000000 and t=0.002001
INFO: Added 20 refined points between t=0.000000 and t=0.002001
INFO: Added 20 refined points between t=1.999345 and t=2.004787
AvgRespTfromCDF =
[[1.00393363 1.00393363 0.        ]
 [1.00393363 4.03663613 0.        ]]
SqCoeffOfVariationRespTfromCDF =
[[1.00395948 1.00395948        nan]
 [1.00395948 0.51511866        nan]]


/tmp/ipykernel_2124599/212424586.py:34: RuntimeWarning: invalid value encountered in scalar divide
  SqCoeffOfVariationRespTfromCDF[i, c] = Variance_R[i, c] / (AvgRespTfromCDF[i, c] ** 2)


In [ ]:
# Plot the CDF response times (excluding Class 3 which has 0 jobs)
import matplotlib.pyplot as plt

# Only plot Classes 1 and 2 (exclude Class 3)
num_classes_to_plot = 2
fig, axes = plt.subplots(model.get_number_of_stations(), num_classes_to_plot, figsize=(12, 8))

for i in range(model.get_number_of_stations()):
    for c in range(num_classes_to_plot):  # Only plot first 2 classes
        if FC[i][c] is not None and len(FC[i][c]) > 0:
            # Data format is [CDF_value, time]
            cdf_values = FC[i][c][:, 0]     # CDF values (column 0)
            time_values = FC[i][c][:, 1]    # Time values (column 1)
            
            # Normal CDF plot for classes with jobs
            axes[i, c].semilogx(time_values, cdf_values, 'b-', label='fluid-steady')
            axes[i, c].legend(loc='best')
            axes[i, c].set_ylabel('F(t) [CDF]')
            axes[i, c].set_xlabel('Time')
            axes[i, c].grid(True, alpha=0.3)
            axes[i, c].set_title(f'CDF: {node[i].name}, {jobclass[c].name}')
        else:
            # No data for this station/class combination
            axes[i, c].text(0.5, 0.5, 'No data', ha='center', va='center', 
                          transform=axes[i, c].transAxes, fontsize=12)
            axes[i, c].set_title(f'{node[i].name}, {jobclass[c].name}')
            axes[i, c].set_xlabel('Time')
            axes[i, c].set_ylabel('F(t) [CDF]')

plt.tight_layout()
plt.show()